# Flow estimation using the calibrated parameters - BPR

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
import pandas as pd


def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

## 1 Load data

In [2]:
df_n = pd.read_csv(ROOT_dir + '/dbs/flow2m.csv')
df_n = df_n.loc[(df_n.Here_Speed_uncap != 0) & (df_n.Here_FFS > df_n.Here_Speed_uncap)]
df_para = pd.read_csv(ROOT_dir + '/results/params.csv')
para_dict = {row['here_seg']: (row['alpha'], row['beta']) for _, row in df_para.iterrows()}

## 2 Compute flow with the calibrated parameters for BPR

In [3]:
def flow_estimator(row):
    ca = row['capacity']
    u_f = row['Here_FFS']
    u = row['Here_Speed_uncap']
    alpha = para_dict[row['Here_segmentID']][0]
    beta = para_dict[row['Here_segmentID']][1]
    return ca * ( (u_f / u - 1) / alpha )**(1/beta)

# df_n.capacity * ( (df_n.Here_FFS / df_n.Here_Speed_uncap - 1) / alpha )**(1/beta)
df_n.loc[:, 'flow'] = df_n.apply(lambda row: flow_estimator(row), axis=1)
df_n.loc[:, 'day'] = df_n.loc[:, 'Time'].apply(lambda x: x.split(' ')[0])
df_n.loc[:, 'hour'] = df_n.loc[:, 'Time'].apply(lambda x: int(x.split(' ')[1].split(':')[0]))

In [7]:
df_n.to_csv(ROOT_dir + '/dbs/flow2m_estimated.csv', index=False)